In [4]:
# Importings base libraries
import os;
import pandas as pd; 
import librosa;
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt


%matplotlib inline


In [5]:
# Input Variables and Setup
training_data_path   = os.path.join(".","dados","TREINAMENTO")
validation_data_path = os.path.join(".","dados","VALIDACAO")


In [44]:
def get_files_from(path):
    from os.path import isfile, join
    from os import listdir
    onlyfiles = [join(path,f) for f in listdir(path) if (isfile(join(path, f)) and f.endswith(".wav")) ]
    return onlyfiles

def load_wav_file(file, intervalo_segundos = 2):
    audio_data, sampling_rate = librosa.load(file, None)
    duracao = audio_data.shape[0]/sampling_rate
    frames_por_audio = sampling_rate * intervalo_segundos
    dados_por_amostra = list()
    for i, ini in enumerate(range(0, audio_data.shape[0], frames_por_audio)):
        dados_por_amostra.append(pd.Series(audio_data[ini:(ini+frames_por_audio)]))
    
    return(dados_por_amostra)

def get_labels(file):
    base = os.path.basename(file)
    without_extension = os.path.splitext(base)[0]
    return without_extension

def get_sampling_rate(file):
    return librosa.load(file, None)[1]

def join_onda_with_label(waves, labels, sampling_rate, duracao_minima = 2.0, duracao_maxima = 2.0):
    labs = pd.Series(list(labels))
    ondas = pd.DataFrame(waves)
    duracoes = [ w.shape[0]/sr for w in waves]
    ondas.insert(loc  = 0, column = 'label', value = labs)
    ondas.insert(loc  = 1, column = 'duracao', value = duracoes)
    ondas.insert(loc = 2, column = 'sr', value = sampling_rate )
    ondas.insert(loc = 3, column = "original_file", value = labels)
    ondas = ondas[ (ondas['duracao'] >= duracao_minima) & (ondas['duracao'] <= duracao_maxima) ]
    return ondas

def get_audio_and_sr_from_df(df, row_index, sample_rate_col_index = 2, first_data_col_index = 4, asNumpyArray = True ):
    data = df.iloc[row_index, first_data_col_index: ].astype('float64')
    if(asNumpyArray):
        data = data.values
    sample_rate = df.iloc[row_index, sample_rate_col_index]
    return(data, sample_rate)

In [46]:
f = get_files_from(training_data_path)
lista_df = list()
processed = 0
to_be = len(f)
for file in f:
    print(f"{file}: Processing {processed} of {to_be}.")
    wave = load_wav_file(file)
    sr = get_sampling_rate(file)
    labels = get_labels(file)
    temp_df = join_onda_with_label(wave, labels, sr)
    lista_df.append(temp_df)
    processed += 1

.\dados\TREINAMENTO\66ah.wav: Processing 0 of 356.
.\dados\TREINAMENTO\66da.wav: Processing 1 of 356.
.\dados\TREINAMENTO\66dc.wav: Processing 2 of 356.
.\dados\TREINAMENTO\66dd.wav: Processing 3 of 356.
.\dados\TREINAMENTO\66h6.wav: Processing 4 of 356.
.\dados\TREINAMENTO\67aa.wav: Processing 5 of 356.
.\dados\TREINAMENTO\67d6.wav: Processing 6 of 356.
.\dados\TREINAMENTO\67hm.wav: Processing 7 of 356.
.\dados\TREINAMENTO\67x6.wav: Processing 8 of 356.
.\dados\TREINAMENTO\6a66.wav: Processing 9 of 356.
.\dados\TREINAMENTO\6acc.wav: Processing 10 of 356.
.\dados\TREINAMENTO\6acd.wav: Processing 11 of 356.
.\dados\TREINAMENTO\6add.wav: Processing 12 of 356.
.\dados\TREINAMENTO\6ahh.wav: Processing 13 of 356.
.\dados\TREINAMENTO\6amh.wav: Processing 14 of 356.
.\dados\TREINAMENTO\6b6m.wav: Processing 15 of 356.
.\dados\TREINAMENTO\6bbm.wav: Processing 16 of 356.
.\dados\TREINAMENTO\6bbx.wav: Processing 17 of 356.
.\dados\TREINAMENTO\6bmb.wav: Processing 18 of 356.
.\dados\TREINAMENTO\6c

.\dados\TREINAMENTO\cd7c.wav: Processing 157 of 356.
.\dados\TREINAMENTO\cdbc.wav: Processing 158 of 356.
.\dados\TREINAMENTO\cdbd.wav: Processing 159 of 356.
.\dados\TREINAMENTO\cdm7.wav: Processing 160 of 356.
.\dados\TREINAMENTO\cdma.wav: Processing 161 of 356.
.\dados\TREINAMENTO\cdmm.wav: Processing 162 of 356.
.\dados\TREINAMENTO\ch66.wav: Processing 163 of 356.
.\dados\TREINAMENTO\ch67.wav: Processing 164 of 356.
.\dados\TREINAMENTO\ch6m.wav: Processing 165 of 356.
.\dados\TREINAMENTO\chb6.wav: Processing 166 of 356.
.\dados\TREINAMENTO\chmx.wav: Processing 167 of 356.
.\dados\TREINAMENTO\chn6.wav: Processing 168 of 356.
.\dados\TREINAMENTO\cm6h.wav: Processing 169 of 356.
.\dados\TREINAMENTO\cmab.wav: Processing 170 of 356.
.\dados\TREINAMENTO\cmhb.wav: Processing 171 of 356.
.\dados\TREINAMENTO\cmmh.wav: Processing 172 of 356.
.\dados\TREINAMENTO\cmnx.wav: Processing 173 of 356.
.\dados\TREINAMENTO\cn67.wav: Processing 174 of 356.
.\dados\TREINAMENTO\cna6.wav: Processing 175 o

.\dados\TREINAMENTO\nhbh.wav: Processing 312 of 356.
.\dados\TREINAMENTO\nhx6.wav: Processing 313 of 356.
.\dados\TREINAMENTO\nmax.wav: Processing 314 of 356.
.\dados\TREINAMENTO\nmdd.wav: Processing 315 of 356.
.\dados\TREINAMENTO\nmhh.wav: Processing 316 of 356.
.\dados\TREINAMENTO\nmhn.wav: Processing 317 of 356.
.\dados\TREINAMENTO\nmnc.wav: Processing 318 of 356.
.\dados\TREINAMENTO\nmnd.wav: Processing 319 of 356.
.\dados\TREINAMENTO\nnbh.wav: Processing 320 of 356.
.\dados\TREINAMENTO\nnxa.wav: Processing 321 of 356.
.\dados\TREINAMENTO\nxdd.wav: Processing 322 of 356.
.\dados\TREINAMENTO\nxmh.wav: Processing 323 of 356.
.\dados\TREINAMENTO\x6m6.wav: Processing 324 of 356.
.\dados\TREINAMENTO\x76c.wav: Processing 325 of 356.
.\dados\TREINAMENTO\x76d.wav: Processing 326 of 356.
.\dados\TREINAMENTO\x77n.wav: Processing 327 of 356.
.\dados\TREINAMENTO\x7hn.wav: Processing 328 of 356.
.\dados\TREINAMENTO\x7na.wav: Processing 329 of 356.
.\dados\TREINAMENTO\xaah.wav: Processing 330 o

In [47]:
df = pd.concat(lista_df, ignore_index=True)
df.to_pickle("training_data.pickle")

In [48]:
df.to_pickle("training_data.pickle")

In [54]:
df.shape

(1414, 88204)

In [65]:
len(df.original_file.unique()) 

356

In [115]:
grupos = df.groupby("original_file").sr.count().sort_values(0)
grupo_problemas = grupos[grupos < 4] 
grupo_problemas

original_file
xxxm    3
dhcd    3
haa6    3
cdbc    3
xbnb    3
cdbd    3
7x6a    3
ca7m    3
6add    3
mdn6    3
Name: sr, dtype: int64

In [ ]:
# Esses arquivos não apresentaram 4 grupos. Hipótese: O arquivo tinha pouco menos de 2 segundos.
# Processá-los manualmente e adicionar ao DF.
# original_file 
# xxxm    3
# dhcd    3
# haa6    3
# cdbc    3
# xbnb    3
# cdbd    3
# 7x6a    3
# ca7m    3
# 6add    3
# mdn6    3
# Name: sr, dtype: int64

In [117]:
# Verificando os que estão no DF - realmente a ultima letra não aparece.
df[[v in grupo_problemas  for v in df["original_file"]]]

In [119]:
# Verificando se podemos obter os dados de audio de uma linha qualquer
data, sr = get_audio_and_sr_from_df(df, 1409)
print(df.label[1409])
ipd.Audio(data, rate=sr)


x


In [120]:
f = get_files_from(validation_data_path)
lista_df_validation = list()
processed = 0
to_be = len(f)
for file in f:
    print(f"{file}: Processing {processed} of {to_be}.")
    wave = load_wav_file(file)
    sr = get_sampling_rate(file)
    labels = get_labels(file)
    temp_df = join_onda_with_label(wave, labels, sr)
    lista_df_validation.append(temp_df)
    processed += 1


.\dados\VALIDACAO\66a6.wav: Processing 0 of 267.
.\dados\VALIDACAO\66ax.wav: Processing 1 of 267.
.\dados\VALIDACAO\66hh.wav: Processing 2 of 267.
.\dados\VALIDACAO\66hn.wav: Processing 3 of 267.
.\dados\VALIDACAO\6767.wav: Processing 4 of 267.
.\dados\VALIDACAO\67bm.wav: Processing 5 of 267.
.\dados\VALIDACAO\6acd.wav: Processing 6 of 267.
.\dados\VALIDACAO\6b7n.wav: Processing 7 of 267.
.\dados\VALIDACAO\6bdc.wav: Processing 8 of 267.
.\dados\VALIDACAO\6bnh.wav: Processing 9 of 267.
.\dados\VALIDACAO\6bxn.wav: Processing 10 of 267.
.\dados\VALIDACAO\6cbb.wav: Processing 11 of 267.
.\dados\VALIDACAO\6cdh.wav: Processing 12 of 267.
.\dados\VALIDACAO\6cmx.wav: Processing 13 of 267.
.\dados\VALIDACAO\6cn7.wav: Processing 14 of 267.
.\dados\VALIDACAO\6d67 (1).wav: Processing 15 of 267.
.\dados\VALIDACAO\6d67.wav: Processing 16 of 267.
.\dados\VALIDACAO\6d7x.wav: Processing 17 of 267.
.\dados\VALIDACAO\6dbb.wav: Processing 18 of 267.
.\dados\VALIDACAO\6ddm.wav: Processing 19 of 267.
.\dado

.\dados\VALIDACAO\dhdc.wav: Processing 163 of 267.
.\dados\VALIDACAO\dhmx.wav: Processing 164 of 267.
.\dados\VALIDACAO\dmbn.wav: Processing 165 of 267.
.\dados\VALIDACAO\dmhh.wav: Processing 166 of 267.
.\dados\VALIDACAO\dmn6.wav: Processing 167 of 267.
.\dados\VALIDACAO\dmxx.wav: Processing 168 of 267.
.\dados\VALIDACAO\dna6 (1).wav: Processing 169 of 267.
.\dados\VALIDACAO\dna6.wav: Processing 170 of 267.
.\dados\VALIDACAO\dndn.wav: Processing 171 of 267.
.\dados\VALIDACAO\dnxm.wav: Processing 172 of 267.
.\dados\VALIDACAO\dxmb.wav: Processing 173 of 267.
.\dados\VALIDACAO\h66x.wav: Processing 174 of 267.
.\dados\VALIDACAO\h6nd.wav: Processing 175 of 267.
.\dados\VALIDACAO\h7xn.wav: Processing 176 of 267.
.\dados\VALIDACAO\ha76.wav: Processing 177 of 267.
.\dados\VALIDACAO\haaa.wav: Processing 178 of 267.
.\dados\VALIDACAO\hamh.wav: Processing 179 of 267.
.\dados\VALIDACAO\hamm (1).wav: Processing 180 of 267.
.\dados\VALIDACAO\hamm.wav: Processing 181 of 267.
.\dados\VALIDACAO\han6.

In [122]:
df_val = pd.concat(lista_df, ignore_index=True)
df_val.to_pickle("validation.pickle")